In [ ]:
import requests
from bs4 import BeautifulSoup
for i in range(0, 999999999):
    # 00 시간 00 분 00 초 000?
    for j in range(24):
        for k in range(60):
            for l in range(24):
                for m in range(999):
                    res = str(j).zfill(2) + str(k).zfill(2) + str(l).zfill(2)+ str(m).zfill(3)
                    
                    url = "https://www.bigkinds.or.kr/news/detailView.do?docId=01100101.20200617" + res
                    raw = requests.get(url, headers={'User-Agent' : 'Mozilla/5.0'})
                    html = BeautifulSoup(raw.text, 'html.parser')
                    if '미분류' in html.text:
                        print(res)
                    else:
                        print(url)

In [ ]:
"""
쌩판 아이디로 하기 힘들다. 수 범위가 너무 큼.
동적 크롤링으로 하기.
동적 크롤링으로 해서 페이지 100페이지로 설정해서 년도별로 수집하기.
수집 정보는 id를 클릭해 얻을 수 있는 url을 일단 전부 한 칸에 넣어서 나중에 칼럼화 하자.
"""


In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver

driver = webdriver.Chrome("./chromedriver")

driver.get("https://www.bigkinds.or.kr/")

#search_box = driver.find_element_by_css_selector("input#total-search-key")
#search_box.send_keys("여성")
# 검색버튼 누르기 // 검색버튼: button.spm
#search_button = driver.find_element_by_css_selector(
#    "button.btn main-search__search-btn news-search-btn")
#search_button.click()

In [ ]:
"""
bigkinds에 주제 태그 등이 있지만 응답 시간이 너무 길어서 크롤링이 힘들다. 
네이버로 해야 겠다.
"""

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

start_time = time.time()

result_list=[]
new_dict=[]

for n in range(1, 1001, 10):
    raw = requests.get("https://search.naver.com/search.naver?where=news&query=여성&sort=1&start="+str(n),
                      headers={'User-Agent':'Mozilla/5.0'}, verify=False)
    html = BeautifulSoup(raw.text, "html.parser")
                   
    articles = html.select("ul.type01 > li")
    
    for ar in articles:
        title = ar.select_one("a._sp_each_title").text
        href = ar.select_one("a").get('href')
        source = ar.select_one("span._sp_each_source").text
        
        try:
            h_raw = requests.get(href, headers={'User-Agent':'Mozilla/5.0'}, verify=False)
            h_html = BeautifulSoup(h_raw.text, "html.parser")
        except:
            h_html = "no text"
            
        new_dict={
            'title': title,
            'source': source,
            'text': h_html,
        }
        result_list.append(new_dict)
        
    print(n)
    
df = pd.DataFrame(result_list)        
txt_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'0630_estimate_size'}.xlsx")
excel_writer = pd.ExcelWriter(txt_name, engine='xlsxwriter')
df.to_excel(excel_writer)
excel_writer.save()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
from openpyxl import load_workbook
#from multiprocessing import Pool
from time import sleep

start_time = time.time()
cal = calendar.Calendar()
result_list=[]
new_dict=[]

query = "여성"

def scrape(url):
    import requests
    from bs4 import BeautifulSoup
    try:
        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'}, allow_redirects=False)
        return res.text
    except:
        return "no text"

if __name__ == '__main__':
    
    
    book = load_workbook('xlsx#1_2019_여성.xlsx')
    writer = pd.ExcelWriter('xlsx#1_2019_여성.xlsx', engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}            
                
    for month in range(1, 13):
        monthdays = [d for d in cal.itermonthdays(2019, month) if d != 0]
        for day in monthdays:
            s_date = "2019." + str(month).zfill(2) + "." + str(day).zfill(2)
            e_date = s_date 
            s_from = s_date.replace(".","")
            e_to = s_from
            print(s_date)

            for n in range(1, 4001, 10):
                url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(n)
                raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
                html = BeautifulSoup(raw.text, "html.parser")
                articles = html.select("ul.type01 > li")

                if not articles:
                    break
                
                title = list()
                urls = list()
                source = list()
                date = list()
                records = list()
                html = list()
                
                for ar in articles:

                    title.append(ar.select_one("a._sp_each_title").text)
                    urls.append(ar.select_one("a").get('href'))
                    source.append(ar.select_one("span._sp_each_source").text)
                    dt = str(ar.select_one("span.bar").next_sibling)
                    if dt:
                        date.append(dt)
                    else:
                        date.append(s_from + ".")
                
                with Pool(10) as p:
                    records = p.map(scrape, urls)
                
                for re in records:
                    html.append(BeautifulSoup(re, "lxml"))
                    
                d = pd.DataFrame({"title": title, "source": source, "date": date, "html": html}) 
                sleep(0.001)
                    
                #book = load_workbook('xlsx#1_2019_여성.xlsx')
                #writer = pd.ExcelWriter('xlsx#1_2019_여성.xlsx', engine='openpyxl')
                #writer.book = book
                #writer.sheets = {ws.title: ws for ws in book.worksheets}

                for sheetname in writer.sheets:
                    d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                
    writer.save()
                
print("--- %s seconds ---" % (time.time() - start_time))

2019.01.01
2019.01.02
2019.01.03


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
from openpyxl import load_workbook
#from multiprocessing import Pool
from collections import ChainMap
from time import sleep

start_time = time.time()
cal = calendar.Calendar()
result_list=[]
new_dict=[]

query = "여성"

#def scrape(url):
#    import requests
#    from bs4 import BeautifulSoup
#    try:
#        res = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
        #html = BeautifulSoup(res, "lxml")
#        return res.text
#    except requests.exceptions.ConnectionError:
#        return "no text"

if __name__ == '__main__':
    for month in range(1, 2):#13):
        monthdays = [d for d in cal.itermonthdays(2019, month) if d != 0]
        for day in monthdays:
            s_date = "2019." + str(month).zfill(2) + "." + str(day).zfill(2)
            e_date = s_date 
            s_from = s_date.replace(".","")
            e_to = s_from
            print(s_date)

            for n in range(1, 10, 10):
                url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(n)
                raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
                html = BeautifulSoup(raw.text, "html.parser")
                articles = html.select("ul.type01 > li")

                if not articles:
                    break
                
                title = list()
                urls = list()
                source = list()
                date = list()
                records = list()
                html = list()
                
                for ar in articles:

                    title.append(ar.select_one("a._sp_each_title").text)
                    urls.append(ar.select_one("a").get('href'))
                    source.append(ar.select_one("span._sp_each_source").text)
                    print(type(ar.select_one("span.bar").next_sibling))
                    if ar.select_one("span.bar").next_sibling is not None:
                        date.append(ar.select_one("span.bar").next_sibling)
                    else:
                        date.append(s_from)
                
                #with Pool(10) as p:
                #    records = p.map(scrape, urls)
                
                #for re in records:
                #    html.append(BeautifulSoup(re, "lxml"))
                for u in urls:
                    try:
                        r = requests.get(u, headers={'User-Agent':'Mozilla/5.0'})
                        html.append(BeautifulSoup(r.text, "html.parser"))
                    except requests.exceptions.ConnectionError:
                        html.append("no text")
                
                d = pd.DataFrame({"title": title, "source": source, "date": date, "html": html}) 
                sleep(0.001)
                    
                book = load_workbook('test2.xlsx')
                writer = pd.ExcelWriter('test2.xlsx', engine='openpyxl')
                writer.book = book
                writer.sheets = {ws.title: ws for ws in book.worksheets}

                for sheetname in writer.sheets:
                    d.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)
                
                writer.save()
                
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from pathos.multiprocessing import ProcessingPool as Pool
from os import getpid
import requests 
def double(i):
    import requests
    from bs4 import BeautifulSoup
    res = requests.get(i)
    html = BeautifulSoup(res.text, "html.parser")
    return res.text#(res.status_code, res.url)
    #print("I'm process")
    #return i * 2
res = requests.get("http://naver.com")

urls = ["http://naver.com", "http://naver.com"]
if __name__ == '__main__':
    with Pool() as pool:
        result = pool.map(double, urls)
        print(result)

In [ ]:
import multiprocessing 
from itertools import repeat 
def a(x, d): 
    d[x] = True 
    
num_cores = multiprocessing.cpu_count() 
# cpu core 개수 
pool = multiprocessing.Pool(num_cores) 
manager = multiprocessing.Manager() 
d = manager.dict() # 프로세스간 공유할 shared dictionary 
input_list = range(0, 10) # 프로세스들에 나눠들어갈 input list 
pool.starmap(a, zip(input_list, repeat(d))
pool.close()
pool.join


In [ ]:
import multiprocessing as mp

def square(x):
    return x**2

if __name__ == '__main__':
    pool = mp.Pool(4)
    results=pool.map(square,range(1,20))

In [ ]:
from multiprocessing import Process, Queue, current_process
import requests
from bs4 import BeautifulSoup
from random import randint
import sys
#sys.setrecursionlimit(1000)

class Spider(object):
    """docstring for Spider"""

    # define a example function
    @staticmethod
    def rand_string(length, output):

        print("{} entry point".format(current_process().name))
        random_post=randint(1000000,9999999)
        response=requests.get('https://stackoverflow.com/questions/'+str(random_post))
        print("{} got request response".format(current_process().name))
        soup=BeautifulSoup(response.content,'lxml')
        try:
            title = soup.find('a',{'class':'question-hyperlink'}).string
        except:
            title = "not found"

        print("{} got title: '{}' of type: {}".format(current_process().name, title, type(title)))

        ###### This did it ######
        title = str(title) #fix or fake news?

        output.put([title,current_process().name])
        output.close()
        print("{} exit point".format(current_process().name))


    # Setup a list of processes that we want to run
#    @staticmethod
    def run(self, outq):
        processes = []
        for x in range(5):
                processes.append(Process(target=self.rand_string, name="process_{}".format(x), args=(x, outq,),) )
                print("creating process_{}".format(x))

        for p in processes:
            p.start()
            print("{} started".format(p.name))

        # Exit the completed processes
        for p in processes:
            p.join()
            print("successuflly joined {}".format(p.name))

        # Get process results from the output queue
        print("joined all workers")
#        return None
        out = []
        while not outq.empty():
            result = outq.get()
            print("got {}".format(result))
            out.append(result)
        return out

# Run processes
if __name__ == '__main__':
    outq = Queue()
    spider=Spider()
    out = spider.run(outq)
    print("done")

In [ ]:
from pathos.multiprocessing import ProcessingPool as Pool
import requests
from bs4 import BeautifulSoup

base_url = 'http://quotes.toscrape.com/page/'

all_urls = list()

def generate_urls():
    for i in range(1,11):
        all_urls.append(base_url + str(i))
    
def scrape(url):
    import requests
    try:
        res = requests.get(url)
        return res.status_code, res.url
    except:
        return "no"

generate_urls()

p = Pool(10)
print(all_urls)
print(p.map(scrape, all_urls))
p.close()
p.join()

In [ ]:
 raw = requests.get("https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=032&listType=title&date="+str(date)+"&page="+str(page),
                           headers={"User-Agent":"Mozilla/5.0"})